<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorGrading/tests/LPTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Color grading via LPTN

[https://github.com/csjliang/LPTN](https://github.com/csjliang/LPTN)


In [1]:
# we'll link the dataset from next-cloud
import os
!apt install -y davfs2  > /dev/null

nextcloud = "/content/nextcloud"
Nextcloud_URL = "https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/" #@param {type:"string"}

os.makedirs(nextcloud, exist_ok=True)


if os.path.isfile("/etc/fstab"):
  os.remove("/etc/fstab")
with open("/etc/fstab" , "a") as f:
  f.write(Nextcloud_URL + " " + nextcloud + " davfs user,rw,auto 0 0")
!mount {nextcloud}

# open existing text file
txt = "/etc/fstab"
viewTxt = open(txt, "r")
txtContent = viewTxt.read()
print("txtContent: {}".format(txtContent))



Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: colab
Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  
/sbin/mount.davfs: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ is already mounted on /content/nextcloud
txtContent: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ /content/nextcloud davfs user,rw,auto 0 0


## 2. create I/O folders

In [2]:
import shutil
from google.colab import files

def create_io(database, topic, library):
  # clean and rebuild the image folders
  input_folder = database+'/'+topic
  # '/content/nextcloud/colorGrading'
  # if os.path.exists(input_folder):
  #   shutil.rmtree(input_folder)
  # os.makedirs(input_folder)

  output_folder = database+'/'+topic+'/'+library# '/content/nextcloud/colorGrading/output'
  if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
  os.makedirs(output_folder)
  return input_folder, output_folder

input_folder, output_folder = create_io('/content/nextcloud','colorGrading','LPTN')

In [3]:
os.listdir(input_folder)

['LPTN',
 '.ipynb_checkpoints',
 'LB_0001.jpg',
 'LB_0002.jpg',
 'LB_0003.jpg',
 'LB_0004.jpg',
 'LB_0005.jpg',
 'LB_0006.jpg',
 'LB_0007.jpg',
 'LB_0008.jpg',
 'LB_0009.jpg',
 'LB_0010.jpg',
 'LB_0011.jpg',
 'LB_0012.jpg',
 'LB_0013.jpg',
 'LB_0014.jpg',
 'LB_0015.jpg',
 'LB_0016.jpg',
 'LB_0017.jpg',
 'LB_0018.jpg',
 'LB_0019.jpg',
 'LB_0020.jpg',
 'LB_0021.jpg',
 'LB_0022.jpg',
 'LB_0023.jpg',
 'LB_0024.jpg',
 'LB_0025.jpg',
 'LB_0026.jpg',
 'LB_0027.jpg',
 'LB_0028.jpg',
 'LB_0029.jpg',
 'LB_0030.jpg',
 'LB_0031.jpg',
 'LB_0032.jpg',
 'LB_0033.jpg',
 'LB_0034.jpg',
 'LB_0035.jpg',
 'LB_0036.jpg',
 'LB_0037.jpg',
 'LB_0038.jpg',
 'LB_0039.jpg',
 'LB_0040.jpg',
 'LB_0041.jpg',
 'LB_0042.jpg',
 'LB_0043.jpg',
 'LB_0044.jpg',
 'LB_0045.jpg',
 'LB_0046.jpg',
 'LB_0047.jpg',
 'LB_0048.jpg',
 'LB_0049.jpg',
 'LB_0050.jpg']

In [4]:
print(output_folder)

/content/nextcloud/colorGrading/LPTN


In [5]:
%ls

LPTN/  nextcloud/  sample_data/


## 3. Clone the Git repository

In [6]:
# clone the repository
%cd /content
if not os.path.exists('LPTN'):
  !git clone https://github.com/csjliang/LPTN
%cd LPTN/

/content
/content/LPTN


## 4. install dependencies

In [7]:
!pip install -r requirement.txt

## 5. Pretrained model download

In [8]:
# dowload the pre-trained model for colorgrading
%cd /content/LPTN/experiments/

# check first if exist, otherwise create path
pretrained_pth = 'pretrained_models/net_g_FiveK_numhigh3.pth'

if not os.path.exists(pretrained_pth):
  !gdown --id 11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml \
          -O pretrained_models/net_g_FiveK_numhigh3.pth

/content/LPTN/experiments


## 6. install pyYAML and update the YAML file with correct path for pretrained_models

In [9]:
!pip install pyYAML

import yaml

# dowload the pre-trained model for colorgrading
%cd /content/LPTN/options/test/LPTN/
 
def read_yaml():
    """ A function to read YAML file"""
    with open('test_FiveK.yml') as f:
        config = yaml.safe_load(f)
 
    return config

read_yaml()

/content/LPTN/options/test/LPTN


{'datasets': {'test': {'dataroot_gt': 'datasets/FiveK/FiveK_test_target.lmdb',
   'dataroot_lq': 'datasets/FiveK/FiveK_test_source.lmdb',
   'io_backend': {'type': 'lmdb'},
   'name': 'FiveK_val',
   'type': 'PairedImageDataset'}},
 'manual_seed': 10,
 'model_type': 'LPTNTestModel',
 'name': 'LPTN_FiveK_480p',
 'network_g': {'nrb_high': 3, 'nrb_low': 5, 'num_high': 3, 'type': 'LPTN'},
 'num_gpu': 1,
 'path': {'pretrain_network_g': 'experiments/pretrained_models/net_g_FiveK_numhigh3.pth',
  'strict_load_g': False},
 'val': {'metrics': {'psnr': {'crop_border': 0,
    'test_y_channel': False,
    'type': 'calculate_psnr'}},
  'save_img': True,
  'suffix': None}}

## 7. dataset downloaden

download dataset

In [21]:
#PYTHONPATH="./:${PYTHONPATH}" python scripts/data_preparation/download_datasets.py
%cd /content/LPTN/scripts/data_preparation/
%ls
!python3 download_datasets.py

/content/LPTN/scripts/data_preparation
create_lmdb.py  download_datasets.py
Traceback (most recent call last):
  File "download_datasets.py", line 6, in <module>
    from codes.utils.download_util import download_file_from_google_drive
ModuleNotFoundError: No module named 'codes'


create lmdb

In [19]:
#PYTHONPATH="./:${PYTHONPATH}" python scripts/data_preparation/create_lmdb.py
!python3 /content/LPTN/scripts/data_preparation/create_lmdb.py

Traceback (most recent call last):
  File "/content/LPTN/scripts/data_preparation/create_lmdb.py", line 1, in <module>
    from codes.utils import scandir
ModuleNotFoundError: No module named 'codes'


## 8. Evaluation

yml file test

In [17]:
# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test.py -opt options/test/LPTN/test_FiveK.yml
!python3 /content/LPTN/codes/test.py -opt options/test/LPTN/test_FiveK.yml

Traceback (most recent call last):
  File "/content/LPTN/codes/test.py", line 5, in <module>
    from codes.data import create_dataloader, create_dataset
ModuleNotFoundError: No module named 'codes'


speed testing

In [18]:
# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml
!python3 /content/LPTN/codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml

Traceback (most recent call last):
  File "/content/LPTN/codes/test_speed.py", line 5, in <module>
    from codes.data import create_dataloader, create_dataset
ModuleNotFoundError: No module named 'codes'
